In [2]:
import os, operator, nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import networkx as nx

In [3]:
f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.txt', 'r')
text = f.read()

In [4]:
# tokenizing the text and filtering nouns / verbs etc
tokens = nltk.word_tokenize(text)
tokens_non_stop = [word for word in tokens if word not in stopwords.words('english')]

def filter_words(key):
    return True if key.startswith('NN') or key.startswith('JJ') or key.startswith('VB') else False
filt_text = map(lambda key: key[0], filter(lambda key: filter_words(key[1]), pos_tag(tokens_non_stop)))

In [5]:
# creating local maps for nodes and edges
dict_text = defaultdict(int)
for t in filt_text:
    dict_text[t] += 1
edges = []
nodes = {}
nodes_number = {}
c = 0
for d in dict_text:
    if d in nodes:
        continue
    nodes[d] = c
    nodes_number[c] = d
    c += 1

In [6]:
# create edges for graph-of-words
edges1=[]
k = 5 ## window size
for i in range(len(filt_text)-k):
    for j in range(i+1, i+k+1):
        wt = j-i  ## this is the weight of the edge (distance between words)
        edges.append((nodes[filt_text[i]], nodes[filt_text[j]], 1.0/wt))
        edges1.append((nodes[filt_text[i]], nodes[filt_text[j]]))
        edges1.append((nodes[filt_text[j]], nodes[filt_text[i]]))

In [7]:
# create graph-of-words
G=nx.Graph()

### TODO: read from file to add edges
G.add_edges_from(edges1)

In [8]:
degree={}
for node in G:
    degree[node]=G.degree(node)    

In [9]:
fw = open('Dataset/edges_celf.txt', 'w')
for line in edges1:
    fw.write("%s\n" % (str(line[0])+" "+str(line[1])+" "+str(1.0/degree[line[1]])))
    

In [10]:
output={}
file = open('Dataset/IC_CelfPlus_Greedy.txt', 'r')
for line in file:
    line=line.split()
    output[line[0]]=line[2]
    

In [11]:
import operator
sorted_output = sorted(output.items(), key=operator.itemgetter(1), reverse=True)

In [12]:
keywords = []
for item in sorted_output:
    keywords.append(nodes_number[int(item[0])])
#     print nodes_number[int(item[0])]
print len(keywords)

50


In [13]:
def computeMetric(origKeys, predKeys):
    n_hits = 0
    n_misses = 0
    n_false_alarms = 0
    for k in predKeys:
        if k in origKeys:
            n_hits += 1
        else:
            n_false_alarms += 1
    for k in origKeys:
        if k not in predKeys:
            n_misses += 1
  
    precision = n_hits*1.0/(n_hits+n_false_alarms)
    recall = n_hits*1.0/(n_hits+n_misses)
  
    F1_score = 2.0*(precision*recall)/(precision+recall)

    print ("Precision = " + str(precision))
    print ("Recall = " + str(recall))
    print ("F1 score = " + str(F1_score))
    #return(list(precision, recall, F1_score))    

def combinePhrases(initKeys, split_text):
    result = []
    len_text = len(split_text)
    for i in xrange(len_text):
        word = split_text[i]
        if word in initKeys:
            combined_word = [word]
            if i + 1 == len_text: result.append(word)   # appends last word if keyword and doesn't iterate
            for j in xrange(i + 1, len_text):
                other_word = split_text[j]
                if other_word in initKeys:
                    combined_word.append(other_word)
                else:
                    for w in combined_word: 
                        if w in initKeys:
                            initKeys.remove(w)
                    result.append(" ".join(combined_word))
                    break
    return result

In [14]:
f = open('Dataset/500N-KPCrowd-v1.1/CorpusAndCrowdsourcingAnnotations/train/art_and_culture-20893614.key', 'r')
origKeys = f.read().split("\n")

In [15]:
#predCelfKeys = combinePhrases(keywords, tokens)

In [16]:
# For CELF
orig_keys = []
for origkey in origKeys:
    orig_keys.extend(origkey.split())
print orig_keys
print keywords

['married', 'Argentine', 'his', 'Grammy', 'ceremony', 'kisses', 'messages', 'romantically', 'vocal', 'Argentine', 'woman', 'Buble', 'dated', 'thanking', 'wedding', 'ceremony', 'Canadian', 'Michael', 'bouquet', 'vocal', 'album', 'Crazy', 'Love', 'Canadian', 'pop', 'Argentina', 'AP', 'mansion', 'Facebook', 'rose', 'Argentine', 'sweetheart', 'month', 'registry', 'South', 'American', 'after', 'tying', 'downtown', 'beautiful', 'traditional', 'Grammy', 'Vancouver', 'chiffon', 'dress', 'Grammywinning', 'family', 'Lopilato', 'made', 'North', 'bride', 'Argentine', 'TV', 'actress', 'media', 'Twitter', 'met', 'high']
['Buble', 'month', 'Aires', 'BUENOS', 'Way', 'bouquet', 'Soul', 'woman', 'Thursday', 'civil', 'Lopilato', 'shouted', 'heels', 'South', 'Photo/Natacha', 'Buenos', 'Angeles', 'actress', 'Grammy-winning', 'several', 'ceremony', 'North', 'Los', 'Rebel', 'mansion', 'Canadian', 'kisses', 'Argentine', 'Michael', 'operas', 'TV', 'years', 'pop', 'guests', 'met', 'star', 'Vancouver', 'sweethea

In [19]:
import string

In [23]:
keywords_new=" ".join(keywords)
keywords_new=keywords_new.translate(None, string.punctuation)
computeMetric(orig_keys,keywords_new.split())

Precision = 0.46
Recall = 0.433962264151
F1 score = 0.446601941748
